In [17]:
import tensorflow as tf
print(tf.version)
import math
import numpy as np
import keras.backend as K
import pandas as pd
from datetime import datetime, date
from sklearn.model_selection import train_test_split

<module 'tensorflow._api.v2.version' from '/home/nbuser/anaconda3_501/lib/python3.6/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [2]:
# @article{scikit-learn,
#  title={Scikit-learn: Machine Learning in {P}ython},
#  author={Pedregosa, F. and Varoquaux, G. and Gramfort, A. and Michel, V.
#          and Thirion, B. and Grisel, O. and Blondel, M. and Prettenhofer, P.
#          and Weiss, R. and Dubourg, V. and Vanderplas, J. and Passos, A. and
#          Cournapeau, D. and Brucher, M. and Perrot, M. and Duchesnay, E.},
#  journal={Journal of Machine Learning Research},
#  volume={12},
#  pages={2825--2830},
#  year={2011}
# }

In [3]:
# import data
train_orig=np.array(pd.read_csv('data/train.csv'))
print(train_orig.shape)
print(train_orig[0:3, :])

(16756, 8)
[[1 nan 'Afghanistan' 33.0 65.0 '2020-01-22' 0.0 0.0]
 [2 nan 'Afghanistan' 33.0 65.0 '2020-01-23' 0.0 0.0]
 [3 nan 'Afghanistan' 33.0 65.0 '2020-01-24' 0.0 0.0]]


In [4]:
#find eastern asia countries/regions
asia=np.array(pd.read_csv('data/mask.csv'))
mask=np.isin(asia[:,3], ['South-Eastern Asia','Eastern Asia'] )
easternAsia=asia[mask][:,1]
easternAsia=np.append(easternAsia, ['Taiwan*'])
print(easternAsia)

['China' 'Indonesia' 'Japan' 'Philippines' 'Vietnam' 'Thailand' 'Myanmar'
 'South Korea' 'Malaysia' 'North Korea' 'Cambodia' 'Laos' 'Singapore'
 'Mongolia' 'Timor-Leste' 'Brunei' 'Taiwan' 'Hong Kong' 'Macao' 'Taiwan*']


In [5]:
# add one binary column eastern asia=1, o.w. 0
train_w_easternAsia=np.c_[train_orig,np.zeros(train_orig.shape[0])] 
print(train_w_easternAsia.shape)
ea_mask=np.isin(train_w_easternAsia[:,2], easternAsia)
train_w_easternAsia[:,8][ea_mask==True]=1
print(train_w_easternAsia[11687, :])

#pre-process dates
train_w_easternAsia[:,5]=train_w_easternAsia[:,5].astype('datetime64',copy=False)
min_date_index=np.argmin(train_w_easternAsia[:,5]) 
min_date=train_w_easternAsia[:,5][min_date_index]
print('min date=', min_date)

train_w_easternAsia_dateEncoded=np.c_[train_w_easternAsia,np.zeros(train_orig.shape[0])] 
train_w_easternAsia_dateEncoded[:,9]=train_w_easternAsia_dateEncoded[:,5]-min_date

for i in range(train_w_easternAsia_dateEncoded[:,9].shape[0]):
    train_w_easternAsia_dateEncoded[i,9]=train_w_easternAsia_dateEncoded[i,9].days+1

print(train_w_easternAsia_dateEncoded[:,9])    

# saving the array
df = pd.DataFrame(train_w_easternAsia_dateEncoded)
df.to_csv('train_w_easternAsia_dateEncoded.csv',index=True)

(16756, 9)
[18420 nan 'South Africa' -30.5595 22.9375 '2020-01-27' 0.0 0.0 0.0]
min date= 2020-01-22
[1 2 3 ... 57 58 59]


In [55]:
#split train/test data
data_x=train_w_easternAsia_dateEncoded[:,[3,4,8,9]]
data_y=train_w_easternAsia_dateEncoded[:,[6,7]]

print("data_x shape=", data_x.shape)
print("data_y shape=", data_y.shape)
print(data_y)

X_train, X_test, Y_train, Y_test = train_test_split(data_x, data_y,test_size=0.2)

X_train=X_train.T
X_test=X_test.T
Y_train=Y_train.T
Y_test=Y_test.T

print("X_train shape=", X_train.shape)
print("X_test shape=", X_test.shape)
print("Y_train shape=", Y_train.shape)
print("Y_test shape=", Y_test.shape)

data_x shape= (16756, 4)
data_y shape= (16756, 2)
[[0.0 0.0]
 [0.0 0.0]
 [0.0 0.0]
 ...
 [2.0 0.0]
 [2.0 0.0]
 [2.0 0.0]]
X_train shape= (4, 13404)
X_test shape= (4, 3352)
Y_train shape= (2, 13404)
Y_test shape= (2, 3352)


In [7]:
#Create placeholders for X and Y. This will allow to later pass training data
def create_placeholders(n_x, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar
    n_y -- scalar
    
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "float"
    Y -- placeholder for the input labels, of shape [n_y, None] and dtype "float"
    
   """
    X=tf.compat.v1.placeholder(float, shape=(n_x, None))
    Y=tf.compat.v1.placeholder(float, shape=(n_y, None))
    return X, Y

In [8]:
# initialize params 
def initialize_parameters():
    """
    Initializes parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [3, 4]
                        b1 : [3, 1]
                        W2 : [2, 3]
                        b2 : [2, 1]
    
    Returns:
    parameters -- a dictionary of tensors containing W1, b1, W2, b2
    """

    parameters={};
#     W1=tf.Variable(name='W1', shape=[3, 4], initializer=tf.initializers.GlorotUniform(seed=1))
#     b1=tf.Variable(name='b1', shape=[3, 1], initializer=tf.zeros_initializer())
    
#     W2=tf.Variable(name='W2', shape=[2, 3], initializer=tf.initializers.GlorotUniform(seed=1))
#     b2=tf.Variable(name='b2', shape=[2, 1], initializer=tf.zeros_initializer())    
    
    W1=tf.compat.v1.get_variable('W1', [3, 4], initializer=tf.compat.v1.glorot_uniform_initializer(seed=1))
    b1=tf.compat.v1.get_variable('b1', [3, 1], initializer=tf.compat.v1.zeros_initializer())    
    
    W2=tf.compat.v1.get_variable('W2', [2, 3], initializer=tf.compat.v1.glorot_uniform_initializer(seed=1))
    b2=tf.compat.v1.get_variable('b2', [2, 1], initializer=tf.compat.v1.zeros_initializer())
    
    W3=tf.compat.v1.get_variable('W3', [2, 2], initializer=tf.compat.v1.glorot_uniform_initializer(seed=1))
    b3=tf.compat.v1.get_variable('b3', [2, 1], initializer=tf.compat.v1.zeros_initializer())    
    
    parameters={"W1": W1,
               "b1": b1,
               "W2": W2,
               "b2": b2,
               "W3": W3,
               "b3": b3 }
    
    return parameters    

In [9]:
tf.compat.v1.reset_default_graph()
with tf.compat.v1.Session() as sess:
    param=initialize_parameters()
print(param)

{'W1': <tf.Variable 'W1:0' shape=(3, 4) dtype=float32>, 'b1': <tf.Variable 'b1:0' shape=(3, 1) dtype=float32>, 'W2': <tf.Variable 'W2:0' shape=(2, 3) dtype=float32>, 'b2': <tf.Variable 'b2:0' shape=(2, 1) dtype=float32>, 'W3': <tf.Variable 'W3:0' shape=(2, 2) dtype=float32>, 'b3': <tf.Variable 'b3:0' shape=(2, 1) dtype=float32>}


In [34]:
#forward prop
def forward_propagation(X, parameters):
    W1=parameters["W1"]
    W2=parameters["W2"]
    W3=parameters["W3"]
    b1=parameters["b1"]
    b2=parameters["b2"]
    b3=parameters["b3"]
    
    Z1=tf.add(tf.matmul(W1, X), b1)
    A1=tf.nn.relu(Z1)
    Z2=tf.add(tf.matmul(W2, A1), b2)
    A2=tf.nn.relu(Z2)
    Z3=tf.add(tf.matmul(W3, A2), b3)
    A3=tf.nn.relu(Z3)   
    
    print("A3=", A3)
    
    return A3

In [35]:
tf.compat.v1.reset_default_graph()
with tf.compat.v1.Session() as sess:
    X, Y=create_placeholders(4, 2)
    Parameters=initialize_parameters()
    A3=forward_propagation(X, Parameters)
    print(A3)

A3= Tensor("Relu_2:0", shape=(2, None), dtype=float32)
Tensor("Relu_2:0", shape=(2, None), dtype=float32)


In [68]:
#define cost function
def compute_cost(A3, Y):
    """
    Computes the cost
    
    Returns:
    cost - Tensor of the cost function
    """
    #get number of training samples
    m = Y.shape[1]
    print("training examples m=", m)
    eps=1e-7
  
    #logdiff = np.multiply(-K.log(A3), Y) + np.multiply(-K.log(1 - A3), 1 - Y)
    
#     sdiff=np.squeeze(np.sum(np.square(np.subtract(A3,Y)), axis=0)) 
#     print("sdiff=", sdiff)
    #sdiff=tf.math.reduce_sum(tf.math.squared_difference(A3,Y),axis=0)
    logdiff = tf.math.multiply_no_nan(-tf.math.log(tf.math.add(A3,eps)), Y) + tf.math.multiply_no_nan(-tf.math.log(tf.math.add(1-A3,eps)), 1 - Y)
    print("logdiff=", logdiff)
    cost=tf.math.reduce_mean(logdiff)
    print("A3=", A3)
    print("Y=", Y)
    print("cost=", cost)
    return cost    

In [69]:
tf.compat.v1.reset_default_graph()
with tf.compat.v1.Session() as sess:
    X, Y=create_placeholders(4, 2)
    Parameters=initialize_parameters()
    A3=forward_propagation(X, Parameters)
    cost=compute_cost(A3,Y)
    print(cost)

A3= Tensor("Relu_2:0", shape=(2, None), dtype=float32)
training examples m= None
logdiff= Tensor("add_5:0", shape=(2, None), dtype=float32)
A3= Tensor("Relu_2:0", shape=(2, None), dtype=float32)
Y= Tensor("Placeholder_1:0", shape=(2, None), dtype=float32)
cost= Tensor("Mean:0", shape=(), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)


In [14]:
def random_mini_batches(X, Y, mini_batch_size=64, seed=0):
    """
    Creates a list of random minibatches from (X, Y)

    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector of shape (output size, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.

    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """

    m = X.shape[1]  # number of training examples
    mini_batches = []
    np.random.seed(seed)

    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0], m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(
        m / mini_batch_size)  # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size: k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size: k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size: m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size: m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    return mini_batches

In [70]:
def model(X_train, Y_train, learning_rate = 0.00005,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    """
    Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->RELU.
    
    Arguments:
    X_train -- training set, of shape (input size = 4, number of training examples)
    Y_train -- test set, of shape (output size = 2, number of training examples)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    tf.compat.v1.reset_default_graph() # to be able to rerun the model without override the variables
    tf.compat.v1.set_random_seed(1)
    seed=3
    costs=[]
    
    (n_X,m)=X_train.shape
    n_Y=Y_train.shape[0]
    
    print("m=", m)
    print("n_X=", n_X)
    print("n_Y=", n_Y)
    
    X,Y=create_placeholders(n_X,n_Y)
    parameters=initialize_parameters()
    A3=forward_propagation(X,parameters)
    cost=compute_cost(A3,Y)
    
    #create optimizer after computing cost
    optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    #define variable initializer
    init=tf.compat.v1.global_variables_initializer()
    
    #make the optimization
    with tf.compat.v1.Session() as sess:
        sess.run(init)
        for i in range(num_epochs):
            epochcost=0.
            seed=seed+1
            num_minibatches=int(m/minibatch_size)
            minibatches=random_mini_batches(X_train,Y_train,minibatch_size,seed)
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y)=minibatch
                _,c=sess.run([optimizer,cost],feed_dict={X:minibatch_X,Y:minibatch_Y})
                print("c=",c)
                print("epochcost=",epochcost)
                epochcost+=c/num_minibatches
            if (i%100==0 and print_cost):
                print("cost after {} thousand: {}".format(i,epochcost))
            if (i%5==0 and print_cost):
                costs.append(epochcost)
                
        plt.title("Learning rate={}".format(learning_rate))
        plt.plot(costs)
        plt.xlabel("iterations(per tens)")
        plt.ylabel("cost")
        plt.show()

        #argmax find the largest element within the matrix
        correct_pred=tf.math.equal(tf.math.argmax(Z3),tf.math.argmax(Y))
        accuracy=tf.reduce_mean(tf.cast(correct_pred,"float"))
        train_accuracy=accuracy.eval({X:X_train,Y:Y_train})
#         test_accuracy=accuracy.eval({X:X_test,Y:Y_test})
        
        print("training accuracy{}".format(train_accuracy))
#         print("testing accuracy{}".format(test_accuracy))
        return parameters         

In [71]:
tf.compat.v1.disable_eager_execution()
parameters=model(X_train,Y_train)

m= 13404
n_X= 4
n_Y= 2
A3= Tensor("Relu_2:0", shape=(2, None), dtype=float32)
training examples m= None
logdiff= Tensor("add_5:0", shape=(2, None), dtype=float32)
A3= Tensor("Relu_2:0", shape=(2, None), dtype=float32)
Y= Tensor("Placeholder_1:0", shape=(2, None), dtype=float32)
cost= Tensor("Mean:0", shape=(), dtype=float32)
c= 425.8703
epochcost= 0.0
c= 332.43573
epochcost= 1.0188284696004037
c= 836.1262
epochcost= 1.8141292590273626
c= 18538.078
epochcost= 3.814431222431968
c= 383.81216
epochcost= 48.16390042099656
c= 140.02596
epochcost= 49.08211134039044
c= 526.35657
epochcost= 49.417101663835886
c= 1388.4227
epochcost= 50.67632790159381
c= 1309.3434
epochcost= 53.99791337884785
c= 537.9414
epochcost= 57.1303138185346
c= 319.59164
epochcost= 58.417254981812114
c= 5746.605
epochcost= 59.18182829350376
c= 331.932
epochcost= 72.92968709845293
c= 687.0338
epochcost= 73.72378280858675
c= 4077.8782
epochcost= 75.3674043719278
c= 140.2778
epochcost= 85.12309378299987
c= 1427.4587
epochcos

c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
cost after 0 thousand: nan
c= nan
epochcost= 0.0
c= na

c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epo

c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epo

c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epo

...

c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epo

c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epo

c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epo

c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan
c= nan
epochcost= nan


KeyboardInterrupt: 

In [ ]:
def predict(X, parameters):
    W1 = tf.convert_to_tensor(parameters["W1"])
    b1 = tf.convert_to_tensor(parameters["b1"])
    W2 = tf.convert_to_tensor(parameters["W2"])
    b2 = tf.convert_to_tensor(parameters["b2"])
    W3 = tf.convert_to_tensor(parameters["W3"])
    b3 = tf.convert_to_tensor(parameters["b3"])

    params = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3}

    x = tf.placeholder("float", [12288, 1])

    z3 = forward_propagation_for_predict(x, params)
    p = tf.argmax(z3)
    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    sess = tf.Session()
    sess.run(init_g)
    sess.run(init_l)
    prediction = sess.run(p, feed_dict={x: X})

    return prediction